## Model Runtime Pipeline

This script performs a single Retrieval-Augmented Generation (RAG) step by:

1. Loading a FAISS index and ID map generated from node embeddings
2. Embedding a user query using a SentenceTransformer model
3. Performing a vector similarity search using FAISS
4. Fetching corresponding node texts from a local JSONL file
5. Building a prompt and passing it to a locally running LLM via Ollama
6. Printing the generated response

Note:
- This script **does not use Neo4j live** — it works with pre-exported data.
- Embeddings and node data must already be generated using `neo4j_exporter.py` and `embed_bge_m3.py`.
- FAISS index must be built once with `build_faiss_index.py`.

### 🧰 Standard Library Modules

- `os`  Handles file path resolution relative to the project structure.

- `json`  Parses ID maps and text data retrieved from FAISS and Neo4j.

---

### 📊 Numerical & Similarity Modules

- `numpy`  Handles vector math and array manipulation for embedding comparisons and FAISS input.

- `faiss`  Facebook AI Similarity Search — performs fast nearest-neighbor search to retrieve the most relevant text embeddings.

- `sklearn.preprocessing.normalize` (imported as `sk_normalize`)   Normalizes query vectors to unit length for cosine similarity search.

---

### 🌐 Networking & Query Modules

- `requests`  Sends the final prompt to a locally hosted LLM via the Ollama server and retrieves the generated response.

---

### 🤖 NLP Module

- `sentence_transformers.SentenceTransformer`  Loads the same embedding model (e.g., BGE-M3) used in pre-processing to embed incoming user queries at runtime.

---

### 🔧 Project-Specific Module

- `digitaiCore.config_loader.ConfigLoader`  Loads configuration settings from `config.yaml` with dot-notation access.
  Ensures consistent access to paths, model info, and runtime behavior throughout the pipeline.Library Stuff

In [12]:
import os
import json
import numpy as np  # For numeric arrays and vector math
import faiss  # Facebook AI Similarity Search - fast vector lookup
import requests  # To send prompt to Ollama server
from sklearn.preprocessing import normalize as sk_normalize  # For cosine similarity
from sentence_transformers import SentenceTransformer  # Used to embed the user query
from digitaiCore.config_loader import ConfigLoader  # Loads config from YAML via dot notation

/Users/alexanderfisher/GitHub/digitai/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Config and pull Key Values💿

In [1]:
repo_root = os.path.abspath(os.path.join(os.path.dirname(__file__), ".."))
config_path = os.path.join(repo_root, "digitaiCore", "config.yaml")
config = ConfigLoader(config_path)

NameError: name 'os' is not defined

In [ ]:
neo4jNodes = os.path.join(repo_root, config.get("dataPaths.neo4jExport")) # File with node text + embeddings
index_path = os.path.join(repo_root, config.get("dataPaths.faissIndex")) # FAISS index file
id_map_path = os.path.join(repo_root, config.get("dataPaths.faissIdMap")) # JSON list mapping FAISS index → node ID
embedding_model = config.get("embedding.model") # SentenceTransformer model
embedding_dim = config.get("vectorIndex.dimension") # Must match FAISS dimensions
normalize = config.get("embedding.normalize") # Whether to normalize for cosine
llm_model = config.get("llm.model") # e.g. "qwen:7b"

## Load FAISS Index 📝 and ID Map 🗺️

In [2]:
print(f"📥 Loading FAISS index from: {index_path}")
index = faiss.read_index(index_path

with open(id_map_path, "r") as f:
    id_map = json.load(f)

NameError: name 'index_path' is not defined

## Load Query Embedding Model 🤖

In [4]:
print(f"🧠 Loading embedding model: {embedding_model}")
model = SentenceTransformer(embedding_model)

NameError: name 'embedding_model' is not defined

## Get User Input 🗣️

In [ ]:
query = input("❓ Enter your query: ").strip()
if not query:
    print("⚠️ No query provided. Exiting.")
    exit()

## Encode the Query 🧮

In [5]:
query_embedding = model.encode(query)

NameError: name 'model' is not defined

## Convert to 2D array and normalize if cosine similarity is enabled 🧮

In [6]:
if normalize:
    query_embedding = sk_normalize([query_embedding], norm="l2")
else:
    query_embedding = np.array([query_embedding])
query_embedding = query_embedding.astype("float32")

NameError: name 'normalize' is not defined

## Perform FAISS Similarity Search

In [ ]:
TOP_K = 5 # Number of top matching texts to retrieve
scores, indices = index.search(query_embedding, TOP_K)

## Filter out invalid results 🧪
    When no natch is returned its set to return -1. By targeting that via the following loop we remove the no match. We dont want to give the no matches to the LLM

In [ ]:
matched_ids = [id_map[i] for i in indices[0] if i != -1]
if not matched_ids:
    print("⚠️ No relevant matches found in the FAISS index.")
    exit()

## Fetch matching texts from JSONL file 📚

In [ ]:
def fetch_node_texts_by_ids(node_ids):
    texts = []
    with open(neo4jNodes, "r", encoding="utf-8") as f:
        for line in f:
            record = json.loads(line)
            if record.get("id") in node_ids and record.get("text"):
                texts.append(record["text"])
    return texts

texts = fetch_node_texts_by_ids(matched_ids)

if not texts:
    print("❌ No node texts found for matched IDs in local file.")
    exit()

## Construct prompt for the LLM

In [7]:
context = "\n".join(f"- {text}" for text in texts)
prompt = f"""You are a chatbot that helps people understand the TEI guidelines which specify how to encode machine-readable texts using XML.

Answer the question below in the **same language the question is asked in**.
Use examples from the provided context as needed — they can be in any language. Do not translate them.

Context:
{context}

Question:
{query}
"""

NameError: name 'texts' is not defined

## Send prompt to locally running LLM

In [8]:
def ask_ollama(prompt, model):
    try:
        response = requests.post(
            "http://localhost:11434/api/generate",
            json={
                "model": model,
                "prompt": prompt,
                "stream": False
            }
        )
        return response.json().get("response", "[ERROR] Empty response from LLM.")
    except Exception as e:
        print("❌ Error while querying Ollama:", e)
        return "[ERROR] Could not get response from local LLM."

## Query the model

In [9]:
print(f"🤖 Sending prompt to LLM ({llm_model})...")
answer = ask_ollama(prompt, llm_model)

NameError: name 'llm_model' is not defined

## Display the result

In [11]:
print("\n🧾 Response:\n")
print(answer)


🧾 Response:



NameError: name 'answer' is not defined